<img src="dataset_5-cover.jpg" alt="Circular Image" style="border-radius: 50%; display: block; margin: 0 auto; width: 200px; height: 200px;">
<h1>Superstore Marketing Campaign Dataset</h1>
<p><i>Sample customer data for analysis of a targeted Membership Offer</i></p>
<h2>About Dataset</h2>
<p><b>Context</b>: A superstore is planning for the year-end sale. They want to launch a new offer - gold membership, that gives a 20% discount on all purchases, for only $499 which is $999 on other days. It will be valid only for existing customers and the campaign through phone calls is currently being planned for them. The management feels that the best way to reduce the cost of the campaign is to make a predictive model which will classify customers who might purchase the offer.</p>
<h2>Data Source:</h2>
<p><b>Kaggle: </b> <a href="https://www.kaggle.com/datasets/ahsan81/superstore-marketing-campaign-dataset">Click Here!!!</a></p>

In [1]:
import duckdb
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

con = duckdb.connect("data/db/superstore.duckdb")

In [2]:
query = """
SHOW TABLES;
"""
con.sql(query).df()

,name
0,campaign_response
1,customer_behavior
2,customer_purchases
3,customers


# 1. Data Profiling Queries

<p>The process for detecting issues (missing data, outliers, inconsistencies) before data is used.</p>


In [3]:
TABLES = ['customers', 'customer_purchases', 'customer_behavior', 'campaign_response']

### 1. Table Schema

In [4]:
# Table Overview
def table_overview(con, tables: List[str]) -> pd.DataFrame:
    overview_data = []
    for table in tables:
        row_count_query = f"SELECT COUNT(*) as row_count FROM {table}"
        row_count = con.execute(row_count_query).fetchone()[0]

        schema_query = f""" 
            SELECT column_name, data_type, is_nullable
            FROM information_schema.columns
            WHERE table_name = '{table}' AND table_schema = 'main'
            ORDER BY ordinal_position;
        """

        schema_df = con.execute(schema_query).df()
        schema_summary = f"{len(schema_df)} columns: {', '.join(schema_df['column_name'].tolist()[:5])}..."
        
        overview_data.append({
            'table': table,
            'row_count': row_count,
            'num_columns': len(schema_df),
            'columns_perview': schema_summary,
            'nullable_cols': ', '.join(schema_df['is_nullable'])
        })
    
    return pd.DataFrame(overview_data)

overview_df = table_overview(con, TABLES)
print("Table Overview")
display(overview_df)

Table Overview


,table,row_count,num_columns,columns_perview,nullable_cols
0,customers,2240,8,"8 columns: Id, Year_Birth, Education, Marital_...","YES, YES, YES, YES, YES, YES, YES, YES"
1,customer_purchases,2240,7,"7 columns: Id, MntWines, MntFruits, MntMeatPro...","YES, YES, YES, YES, YES, YES, YES"
2,customer_behavior,2240,7,"7 columns: Id, Recency, NumDealsPurchases, Num...","YES, YES, YES, YES, YES, YES, YES"
3,campaign_response,2240,3,"3 columns: Id, Response, Complain...","YES, YES, YES"


### 2. Missing Values

In [ ]:
def missing_values_profile(con, tables: List[str]) -> pd.DataFrame:
    missing_data = []
    for table in tables:
        schema = con.execute(f"""
            SELECT column_name FROM information_schema.columns 
            WHERE table_name = '{table}' AND table_schema = 'main'
        """).df()
        
        total_rows_q = f"SELECT COUNT(*) FROM {table}"
        total_rows = con.execute(total_rows_q).fetchone()[0]
        
        for col in schema['column_name']:
            null_query = f"SELECT COUNT(*) - COUNT({col}) as null_count FROM {table}"
            null_count = con.execute(null_query).fetchone()[0]
            null_pct = (null_count / total_rows) * 100 if total_rows > 0 else 0
            
            missing_data.append({
                'table': table,
                'column': col,
                'null_count': null_count,
                'null_percentage': round(null_pct, 2)
            })
    
    return pd.DataFrame(missing_data).sort_values('null_percentage', ascending=False)

In [8]:
missing_data = missing_values_profile(con, TABLES)
print("Missing Values Profiling")
display(missing_data)

Missing Values Profiling


,table,column,null_count,null_percentage
4,customers,Income,24,1.07
0,customers,Id,0,0.00
13,customer_purchases,MntSweetProducts,0,0.00
23,campaign_response,Response,0,0.00
22,campaign_response,Id,0,0.00
21,customer_behavior,NumWebVisitsMonth,0,0.00
20,customer_behavior,NumStorePurchases,0,0.00
19,customer_behavior,NumCatalogPurchases,0,0.00
18,customer_behavior,NumWebPurchases,0,0.00
17,customer_behavior,NumDealsPurchases,0,0.00


<p>The Income column in the customers table has only 1% missing values, or just 24 empty values <strong>not urgent</strong> to handle since it's possible that customers did not fill it in or the income is unknown, and this is not a mandatory thing</p>


### 3. Duplicate Values